In [2]:
!pip install riotwatcher

In [1]:
import requests, json
from riotwatcher import LolWatcher, ApiError
import pandas as pd

In [9]:
api_key="RGAPI-d51ac6d4-0b2b-49f2-9db5-84807339da35"
watcher = LolWatcher(api_key)
name="Silverakai"
ma_region="euw1"
my_region="europe"

me = watcher.summoner.by_name(ma_region, name)
print(me)

{'id': 'C754ragTdklFiKmrlqop3vlAPX6U3fLXTDlyG22wythKil0-tiIEok51-g', 'accountId': '01rHa5QHc2aRaXMwgdCYs9p_RiS61OqB5JluMAiVluin--1Zx45uaF8S', 'puuid': 'v_sTUHg9OBfL0kflx44NGgEf0_Jqk1FTzLG9JBflrmDBR7t8LPKEqtcf8ynl1CmKkheDHdeedIK4jA', 'name': 'Silverakai', 'profileIconId': 5031, 'revisionDate': 1649191160000, 'summonerLevel': 103}


In [10]:
#création url valider recup données

urlbase = "https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" 
urlfinal = urlbase + name + "?api_key=" + api_key
print (urlfinal)

https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Silverakai?api_key=RGAPI-d51ac6d4-0b2b-49f2-9db5-84807339da35


In [11]:
#get les paramètres utilisateurs essentiels id, puuid et accountid

response = requests.get(urlfinal)
response.json()
idinvocateur = response.json()["id"]
acidinvocateur = response.json()["accountId"]
puuidinvocateur = response.json()["puuid"]

In [11]:
#Stats générales en classées

my_ranked_stats = watcher.league.by_summoner(ma_region, me['id'])
for cle, valeur in my_ranked_stats[0].items():
    print(cle,valeur)

leagueId 4146245d-b4ab-4602-8588-a4167f7d0979
queueType RANKED_SOLO_5x5
tier IRON
rank I
summonerId C754ragTdklFiKmrlqop3vlAPX6U3fLXTDlyG22wythKil0-tiIEok51-g
summonerName Silverakai
leaguePoints 47
wins 105
losses 96
veteran False
inactive False
freshBlood False
hotStreak False


In [12]:
#Récupérer liste des matchs

nbmatch="20"

urlbase2 = "https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/"
urlfinal2 = urlbase2 + puuidinvocateur + "/ids?start=0&count=20" + "?api_key=" + api_key 
print (urlfinal2)

NameError: name 'puuidinvocateur' is not defined

In [ ]:
response2 = requests.get(urlfinal2)
response2.json()

In [13]:
#variables à analyser
# maxCsAdvantageOnLaneOpponent, championName, win, totalMinionsKilled, timePlayed, firstBloodKill, visionScore, 
#kills, deaths, assists


my_matches = watcher.match.matchlist_by_puuid(my_region, me['puuid'])
adversaire = "Yasuo" 
k=0 #compteur
victoire = 0 #nb de victoires
fb = 0 #nb de first blood

#Détails de la dernière game -> 0
last_match = my_matches[1]
match_detail = watcher.match.by_id(my_region, last_match)

a=match_detail.get("info") #parcourir le dictionnaire de données
b=a.get("participants")
c=b[2].get("championName")
#print(c)

for i in range(10) : 
    if b[i].get("championName") == adversaire : 
        if b[i].get("win") == True :
            victoire=victoire+1
        if b[i].get("firstBloodKill") == True :
            fb=fb+1
        k=k+1
        wr= "Win rate : " + str(victoire*100/(k)) + "%"
        print(wr)
        
        fbr = "First Blood Rate : " + str(fb*100/(k)) + "%"
        print (fbr)
        
        csing= "CS/min : " + str(b[i].get("totalMinionsKilled")*60/b[i].get("timePlayed"))
        print (csing)
        
        vision = "Vision/min : " + str(b[i].get("visionScore")*60/b[i].get("timePlayed"))
        print (vision)
        
        
#print(match_detail)


In [19]:
my_matches = watcher.match.matchlist_by_puuid(my_region, me['puuid'],0,100)

adversaire = "Syndra" 
k=0 #compteur de games
victoire = 0 #nb de victoires
fb = 0 #nb de first blood
minion = 0 
tpsjeu = 0
vision1 = 0
kills = 0
deaths = 0
assists = 0
            

for i in range(14) :
    last_match = my_matches[i]
    match_detail = watcher.match.by_id(my_region, last_match)

    a=match_detail.get("info") #parcourir le dictionnaire de données
    if a.get("gameType") != "CUSTOM_GAME" :  #si on est en practice 1 partcipant donc out of range
        b=a.get("participants")
    
        for j in range(10) :                                 
            if b[j].get("championName") == adversaire and b[j].get("summonerName") == name : 
                if b[j].get("win") == True :
                    victoire=victoire+1
                if b[j].get("firstBloodKill") == True :
                    fb=fb+1
                k=k+1
                
                kills = kills + b[j].get("kills")
                deaths = deaths + b[j].get("deaths")
                assists = assists + b[j].get("assists")
                minion = minion + b[j].get("totalMinionsKilled")*60 
                tpsjeu = tpsjeu + b[j].get("timePlayed")
                vision1 = vision1 + b[j].get("visionScore")*60
                
                kda = "KDA : " + str(round(kills/k,2)) + "/" + str(round(deaths/k,2)) + "/" + str(round(assists/k,2)) 

                wr= "Win rate : " + str(round(victoire*100/k,2)) + "%"


                fbr = "First Blood Rate : " + str(round(fb*100/k,2)) + "%"


                csing= "CS/min : " + str(round(minion/tpsjeu,2))


                vision = "Vision/min : " + str(round(vision1/tpsjeu,2))
                
        
print(wr)
print(kda)
print (fbr)
print (csing)
print (vision)
del(wr) #fait que le programme plante si aucune game ne correspond aux critères

Win rate : 66.67%
KDA : 6.0/3.33/4.33
First Blood Rate : 0.0%
CS/min : 4.68
Vision/min : 0.42


In [173]:
#bon winrate : Il obitent de bon résultats régulièrement avec ce champion, soyez prudent
#bon farming : essayez de le punir s'il utilise ses sorts pour le farm, ne le laissez pas prendre l'avantage en gold
#bon kda : Il est surement présent sur toute la map, ne le laissez pas roam en jouant sur le farm
#bon first blood : il essaira surment de vous engage très tôt dans la partie
#si l'on obtient exactement les mêmes résultats en changeant de champion, il ne l'a pas joué pour la période donnée ou erreur 
#d'écriture


In [110]:
my_matches = watcher.match.matchlist_by_puuid(my_region, me['puuid'],0,34)
print(my_matches)

['EUW1_5804610087', 'EUW1_5804633209', 'EUW1_5804191163', 'EUW1_5804105603', 'EUW1_5804040385', 'EUW1_5804045220', 'EUW1_5803621860', 'EUW1_5803507267', 'EUW1_5801124504', 'EUW1_5800959421', 'EUW1_5800962617', 'EUW1_5800836412', 'EUW1_5800769314', 'EUW1_5799205976', 'EUW1_5798034391', 'EUW1_5797957377', 'EUW1_5796160800', 'EUW1_5796073632', 'EUW1_5796044847', 'EUW1_5795950696', 'EUW1_5795827258', 'EUW1_5795355816', 'EUW1_5795269771', 'EUW1_5795283007', 'EUW1_5795197082', 'EUW1_5795220824', 'EUW1_5795115439', 'EUW1_5791735339', 'EUW1_5791589349', 'EUW1_5791546355', 'EUW1_5791532276', 'EUW1_5791319197', 'EUW1_5791267708', 'EUW1_5791210715']
